In [ ]:
!pip install -q kaggle
from google.colab import files
uploaded = files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Завантаження свого датасету
!kaggle datasets download -d kovalkostiantyn/lab5-data
!unzip -q lab5-data.zip -d /content/

import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import random
import matplotlib.image as mpimg
import seaborn as sns
from collections import Counter
from google.colab import files 

# --- Константи ---
DATASET_DIR = "/content/train"      # Тренувальний датасет
TEST_DIR = "/content/test"          # Тестовий датасет
IMG_SIZE = (299, 299)               # Розмір зображень для Inception
BATCH_SIZE = 16                     
EPOCHS = 20                        
LR = 1e-4                           
MODEL_PATH = "/content/best_inception_binary.h5"  
CLASS_INDICES_JSON = "/content/class_indices.json"
METADATA_JSON = "/content/metadata.json"

# Створення генераторів даних
def create_generators(dataset_dir=DATASET_DIR):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.12,
        height_shift_range=0.12,
        shear_range=0.12,
        zoom_range=0.12,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )
    
    train_gen = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        shuffle=True
    )
    
    val_gen = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        shuffle=False
    )
    
    # Збереження мапування класів
    with open(CLASS_INDICES_JSON, "w", encoding="utf-8") as f:
        json.dump(train_gen.class_indices, f, ensure_ascii=False, indent=2)
    
    return train_gen, val_gen

# Модуль Inception 
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, name=None):
    # 1x1
    branch1 = layers.Conv2D(filters_1x1, (1,1), padding='same', activation='relu')(x)
    
    # 1x1 -> 3x3
    branch3 = layers.Conv2D(filters_3x3_reduce, (1,1), padding='same', activation='relu')(x)
    branch3 = layers.Conv2D(filters_3x3, (3,3), padding='same', activation='relu')(branch3)
    
    # 1x1 -> 3x3 -> 3x3
    branch5 = layers.Conv2D(filters_5x5_reduce, (1,1), padding='same', activation='relu')(x)
    branch5 = layers.Conv2D(filters_5x5, (3,3), padding='same', activation='relu')(branch5)
    branch5 = layers.Conv2D(filters_5x5, (3,3), padding='same', activation='relu')(branch5)
    
    # Pool -> 1x1
    branch_pool = layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
    branch_pool = layers.Conv2D(filters_pool_proj, (1,1), padding='same', activation='relu')(branch_pool)
    
    return layers.concatenate([branch1, branch3, branch5, branch_pool], axis=-1, name=name)

# Побудова моделі
def build_inception_like(input_shape=(299,299,3), dropout_rate=0.4):
    inp = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(32, (3,3), strides=(2,2), padding='valid', activation='relu')(inp)
    x = layers.Conv2D(32, (3,3), padding='valid', activation='relu')(x)
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='valid')(x)
    
    x = layers.Conv2D(80, (1,1), activation='relu')(x)
    x = layers.Conv2D(192, (3,3), padding='valid', activation='relu')(x)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='valid')(x)
    
    x = inception_module(x, 64, 48, 64, 64, 96, 32, name="incept_1")
    x = inception_module(x, 64, 48, 64, 64, 96, 64, name="incept_2")
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='valid')(x)
    
    x = inception_module(x, 128, 96, 128, 96, 128, 128, name="incept_3")
    x = inception_module(x, 160, 112, 160, 112, 160, 128, name="incept_4")
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(dropout_rate/2)(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=inp, outputs=out, name="InceptionLike_binary")
    return model

# Компіляція та навчання
def compile_and_train(model, train_gen, val_gen, epochs=EPOCHS):
    model.compile(
        optimizer=optimizers.Adam(learning_rate=LR),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    # Балансування класів
    counter = Counter(train_gen.classes)
    majority = max(counter.values())
    class_weight = {cls: float(majority / count) for cls, count in counter.items()}
    
    # Колбеки
    cb = [
        callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1),
        callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_loss', verbose=1)
    ]
    
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        class_weight=class_weight,
        callbacks=cb,
        verbose=1
    )
    
    return history

# Оцінка моделі
def evaluate_model(model, val_gen):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1).ravel()
    y_pred = (preds > 0.5).astype(int)
    y_true = val_gen.classes
    
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    
    print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys()), zero_division=0))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(val_gen.class_indices.keys()),
                yticklabels=list(val_gen.class_indices.keys()))
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()
    
    meta = {"accuracy": float(acc), "precision": float(prec), "recall": float(rec), "f1": float(f1)}
    with open(METADATA_JSON, "w") as f:
        json.dump(meta, f, indent=2)
    
    return {"acc":acc, "prec":prec, "rec":rec, "f1":f1, "cm":cm}

# Графіки навчання
def plot_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    if 'accuracy' in history.history and 'val_accuracy' in history.history:
        axes[0, 0].plot(history.history['accuracy'], label='Train')
        axes[0, 0].plot(history.history['val_accuracy'], label='Validation')
        axes[0, 0].set_title('Accuracy')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
    else:
        axes[0, 0].text(0.5, 0.5, 'Accuracy not found', ha='center', va='center')
        axes[0, 0].set_title('Accuracy')

    # Loss
    if 'loss' in history.history and 'val_loss' in history.history:
        axes[0, 1].plot(history.history['loss'], label='Train')
        axes[0, 1].plot(history.history['val_loss'], label='Validation')
        axes[0, 1].set_title('Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
    else:
        axes[0, 1].text(0.5, 0.5, 'Loss not found', ha='center', va='center')
        axes[0, 1].set_title('Loss')

    # Precision
    if 'precision' in history.history and 'val_precision' in history.history:
        axes[1, 0].plot(history.history['precision'], label='Train')
        axes[1, 0].plot(history.history['val_precision'], label='Validation')
        axes[1, 0].set_title('Precision')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    else:
        axes[1, 0].text(0.5, 0.5, 'Precision not found', ha='center', va='center')
        axes[1, 0].set_title('Precision')

    # Recall
    if 'recall' in history.history and 'val_recall' in history.history:
        axes[1, 1].plot(history.history['recall'], label='Train')
        axes[1, 1].plot(history.history['val_recall'], label='Validation')
        axes[1, 1].set_title('Recall')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    else:
        axes[1, 1].text(0.5, 0.5, 'Recall not found', ha='center', va='center')
        axes[1, 1].set_title('Recall')

    plt.tight_layout()
    plt.show()

# Прогноз на одному зображенні
def predict_image(model, image_path, class_indices_path=CLASS_INDICES_JSON):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    arr = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    
    prob = model.predict(arr, verbose=0)[0,0]
    pred = int(prob > 0.5)
    
    with open(class_indices_path, "r", encoding="utf-8") as f:
        cls = json.load(f)
    inv = {v: k for k, v in cls.items()}
    label = inv.get(pred, str(pred))
    
    return label, float(prob)

# Прогноз на випадкових зображеннях з датасету
def predict_random_from_dataset(model, dataset_dir=DATASET_DIR, num_samples=5):
    if not os.path.exists(dataset_dir):
        print("Папка з датасетом не знайдена!")
        return

    # Завантажуємо мапування класів
    with open(CLASS_INDICES_JSON, "r", encoding="utf-8") as f:
        class_indices = json.load(f)
    index_to_class = {v: k for k, v in class_indices.items()}

    # Збираємо всі файли зображень
    image_paths = []
    true_labels = []
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(class_dir, fname))
                    true_labels.append(class_name)

    if len(image_paths) == 0:
        print("Зображення не знайдено в датасеті!")
        return

    # Випадковий вибір
    indices = random.sample(range(len(image_paths)), min(num_samples, len(image_paths)))
    
    plt.figure(figsize=(15, 4 * ((len(indices) + 2) // 3)))
    
    for i, idx in enumerate(indices):
        img_path = image_paths[idx]
        true_label = true_labels[idx]

        # Завантаження та прогноз
        pred_label, prob = predict_image(model, img_path)
        confidence = prob if pred_label == index_to_class[1] else (1 - prob)
        is_correct = "Правильно" if pred_label == true_label else "Помилка"

        # Відображення
        plt.subplot((len(indices) + 2) // 3, 3, i + 1)
        img = mpimg.imread(img_path)
        plt.imshow(img)
        plt.title(f"Справжнє: {true_label}\n"
                  f"Прогноз: {pred_label} ({confidence:.1%})\n"
                  f"{is_correct}", 
                  color='green' if is_correct == "Правильно" else 'red')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def main():
    if not os.path.exists(DATASET_DIR):
        print("Датасет не знайдено!")
        return
    
    print("Створення генераторів...")
    train_gen, val_gen = create_generators(DATASET_DIR)
    
    print("Побудова моделі...")
    model = build_inception_like()
    model.summary()
    
    print("Навчання моделі...")
    history = compile_and_train(model, train_gen, val_gen, epochs=EPOCHS)
    
    print("Візуалізація результатів...")
    plot_history(history)
    
    print("Оцінка на валідації...")
    evaluate_model(model, val_gen)
    
    # --- Запуск перевірки на випадкових зображеннях ---
    print("Перевірка на зображеннях з датасету(test)")
    best_model = tf.keras.models.load_model(MODEL_PATH)
    predict_random_from_dataset(best_model, dataset_dir=DATASET_DIR, num_samples=6)

# --- Запуск ---
if __name__ == "__main__":
    main()


In [ ]:
# Завантаження фото та прогноз
def upload_and_predict():
    print("Завантажте зображення для класифікації:")
    uploaded = files.upload()
    
    if not uploaded:
        print("Фото не завантажено.")
        return
        
    if not os.path.exists(MODEL_PATH):
        print("Модель не знайдена! Спочатку навчіть модель.")
        return
    
    model = tf.keras.models.load_model(MODEL_PATH)
    
    for filename in uploaded.keys():
        print(f"\nОбробка: {filename}")
        label, prob = predict_image(model, filename)
        confidence = prob if label == list(model.output_names)[0] else (1 - prob)
        print(f"Прогноз: {label} (впевненість: {confidence:.2%})")
upload_and_predict()
